## TP3 : Logistic regression

The purpose of this tutorial is to implement and use the Logistic Regression for binary classification. We will apply this
method to the problem of handwritten characters to learn how to
distinguish two numbers (here 5 and 6).


In [ ]:
from pylab import *
#import numpy as np
#import matplotlib.pyplot as plt

from numpy import linalg as la

## 1. Logistic regression, IRLS algorithm.

### Preliminary question: the algorithm

#### Have a look at the function `regression_logistique.m` and locate the main steps of the algorithm you have been taught (see course).
You can comment the code in the cell below.

In [ ]:
def regression_logistique(X,t,Nitermax=20,eps_conv=1e-3):
    '''Entrees :
    X = [ones(N_train,1) x_train];
    t = class_train 
    Nitermax = nombre maximale d'itérations (20 par défaut)
    eps_conv = critère de convergence sur norm(w-w_old)/norm(w) ; 
    eps_conv=1e-3 par défaut
    
    Sorties : 
    w : vecteur des coefficients de régression logistique
   Niter : nombre d'itérations utilisées effectivement
   
   Fonction de régression logistique pour la classification binaire.
   
   Utilisation :
       Nitermax = 50
       eps_conv = 1e-4
       [w,Niter] = regression_logistique(X,t,Nitermax,eps_conv)
    '''
    N_train = X.shape[0]

    #initialisation : 1 pas de l'algorithme IRLS
    w = np.zeros((X.shape[1],))
    w_old = w 
    y = 1/2*np.ones((N_train,))
    R = np.diag(y*(1-y))   # diag(y_n(1-y_n))
    z = X.dot(w_old)-la.inv(R).dot(y-t)
    w = la.inv(X.T.dot(R).dot(X)).dot(X.T).dot(R).dot(z)

    # boucle appliquant l'algortihme de Newton-Raphson
    Niter = 1
    while ( (la.norm(w-w_old)/la.norm(w)>eps_conv) | (Niter<Nitermax) ):
        Niter = Niter+1
        y = 1/(1+np.exp(-X.dot(w)))
        R = np.diag(y*(1-y))  
        w_old = w 
        z = X.dot(w_old)-la.inv(R).dot(y-t) 
        w = la.inv(X.T.dot(R).dot(X)).dot(X.T).dot(R).dot(z)
         
    return w, Niter


### Reading & preparing synthetic data

Load the training and test data sets `synth_train.txt`
and `synth_test.txt`. The targets t belong to {1,2} and the features  
x belong to R^2. 

We have 100 training samples and 200 test samples

* the 1st column contains the label of each sample, 
* columns 2 and 3 contain the coordinate of each point.

In [ ]:
# Training set
synth_train = np.loadtxt('synth_train.txt') 
class_train = synth_train[:,0]
class_train_1 = np.where(synth_train[:,0]==1)[0]
class_train_2 = np.where(synth_train[:,0]==2)[0]
x_train = synth_train[:,1:]
N_train = np.size(x_train,axis=0)

# Test set
synth_test = np.loadtxt('synth_test.txt')
class_test = synth_test[:,0]
class_test_1 = np.where(synth_test[:,0]==1)[0]
class_test_2 = np.where(synth_test[:,0]==2)[0]
x_test = synth_test[:,1:]
N_test = np.size(x_test,axis=0)


In [ ]:
import plotly.express as px
import pandas as pd

px.scatter(pd.DataFrame(synth_train, columns = ['classe', 'x1', 'x2']),x='x1', y='x2', color='classe', title = 'Data visualization of training dataset')

In [ ]:
px.scatter(pd.DataFrame(synth_test, columns = ['classe', 'x1', 'x2']),x='x1', y='x2', color='classe', title = 'Data visualization of test dataset')

### Preparing features for logistic regression (binary classification)
First, we prepare the feature matrix and the target vector associated to 
the training and test sets:


In [ ]:
X = np.hstack((np.ones((N_train,1)),x_train))
t = 2-class_train   # 0 if class=2, 1 if class=1

X_test = np.hstack((np.ones((N_test,1)),x_test))
t_test = 2-class_test   # 0 if class=2, 1 if class=1

### Exercise 1 : the logistic function of decision

1. Use the function `regression_logistique.m` to estimate the logistic
regression vector `w`. *Indication : use `Nitermax = 50;
eps_conv=1e-3;`.*
2. Compute the decision function $f(x) = argmax_k P(C_k|x)$ on the test set
to get the classification results. Recall that $y_n=\sigma(w^T x)$ (logistic function)
and that *using vectors* you may directly write $y=\sigma(Xw)$, with the
column of ones in X.
3. Display the results by plotting the points from both the training set
and the test set.
4. Write the equation which defines the decision boundary.
5. Artificially add a few points to the training set far from the decision boundary to check the robustness of logistic regression to outliers. Check the behaviour of LDA for comparison in this case and comment. 


In [ ]:
#Exercise 1
w, Niter = regression_logistique(X, t)

In [ ]:
predictions = lambda x: np.where(np.exp(x)/( 1 + np.exp(x))<=0.5, 0 , 1)

In [ ]:
t_train_pred = predictions(X @ w)
t_test_pred = predictions(X_test @ w)

In [ ]:
px.scatter(pd.DataFrame(dict(prediction=2-t_train_pred, x1 = synth_train[:,1], x2 = synth_train[:,2])),x='x1', y='x2', color='prediction', title = 'Data visualization of predictions for training dataset')

In [ ]:
px.scatter(pd.DataFrame(dict(prediction=2-t_test_pred, x1 = synth_test[:,1], x2 = synth_test[:,2])),x='x1', y='x2', color='prediction', title = 'Data visualization of predictions for training dataset')

You can draw the decision boundary $w^Tx = 0$ by using: 


In [ ]:
import plotly.graph_objects as go

# First compute w... then: 
x1 = np.linspace(-2.5,1.5,10) 
x2 = (-w[0]-w[1]*x1)/w[2]
fig1 = px.line(pd.DataFrame(dict(x1=x1, x2=x2)), x='x1', y='x2')
fig2 = px.scatter(pd.DataFrame(dict(prediction=2-t_train_pred, x1 = synth_train[:,1], x2 = synth_train[:,2])),x='x1', y='x2', color='prediction')
fig_boundary = go.Figure(data = fig1.data + fig2.data)
fig_boundary.update_layout(dict(title='Decision Boundary with predictions of training dataset'))
fig_boundary.show()

In [ ]:
fig2_test = px.scatter(pd.DataFrame(dict(prediction=2-t_test_pred, x1 = synth_test[:,1], x2 = synth_test[:,2])),x='x1', y='x2', color='prediction')
fig_boundary_test = go.Figure(data = fig1.data + fig2_test.data)
fig_boundary_test.update_layout(dict(title='Decision Boundary with predictions of test dataset'))
fig_boundary_test.show()

Adding artificial outliers : 

Il ne faut pas ajouter des points trop éloignés des points du dataset sinon la matrice R dans l'implementation de l'algorithme ne sera plus inversible : Pourquoi -> Pour de très grosses valeur on a un sigma très proche de 0 ou de 1 et donc y-t très proche de zero donc plus inversible.

In [ ]:
from scipy.stats import norm
nb_points = 5
outliers_points = np.repeat([[-3,4],[2,-1]],[nb_points,nb_points], axis=0) + norm.rvs(loc = 0, scale = 0.2, size = nb_points*4).reshape((nb_points*2,2))
classe_outliers = np.concatenate((np.full(nb_points,1), np.full(nb_points,2)))

In [ ]:
px.scatter(pd.DataFrame(dict(x1=outliers_points[:,0], x2=outliers_points[:,1], classe=classe_outliers)),
           x='x1',
           y='x2',
           color='classe',
           title = 'Outliers who where be add')

In [ ]:
X_outliers = np.hstack((np.ones((outliers_points.shape[0],1)),outliers_points))
t_outliers = 2-classe_outliers

In [ ]:
w_outliers, Niter_outliers = regression_logistique(np.concatenate((X, X_outliers)), np.concatenate((t, t_outliers)))

In [ ]:
t_train_outliers_pred = predictions(np.concatenate((X, X_outliers)) @ w_outliers)

In [ ]:
x2_outliers = (-w_outliers[0]-w_outliers[1]*x1)/w_outliers[2]

In [ ]:
fig1_outliers = px.line(pd.DataFrame(dict(x1=x1, x2=x2)), x='x1', y='x2')
fig2_outliers = px.scatter(pd.DataFrame(dict(prediction=2-t_train_outliers_pred, x1 = np.concatenate((X, X_outliers))[:,1], x2 = np.concatenate((X, X_outliers))[:,2])),x='x1', y='x2', color='prediction')
fig_boundary_outliers = go.Figure(data = fig1_outliers.data + fig2_outliers.data)
fig_boundary_outliers.update_layout(dict(title='Decision Boundary with predictions of training dataset'))
fig_boundary_outliers.show()

## 2. Application: handwritten digits recognition 5 & 6
We load 2 matrices which contain each a sequence of examples of 16x16 images 
of handwritten digits which are 5 and 6 here. Each line of the matrix
contains 256 pixel values coding for the gray level of a 16x16 image.

In [ ]:
train_5 = np.loadtxt('train_5.txt',delimiter=',')   # 556 samples
train_6 = np.loadtxt('train_6.txt',delimiter=',')   # 664 samples

### Examples of images


In [ ]:
# Digit 5
n=9
I = np.reshape(train_5[n,:],(16,16))

plt.imshow(I,cmap='gray')
plt.show()

In [ ]:
# Digit 6
n=5
I = reshape(train_6[n,:],(16,16))

plt.imshow(I,cmap='gray')
plt.show()

### Separating the training and test sets

We keep in the training set the 145 first images of 5s and the 200 first
images of 6s:


In [ ]:
x_train_brut = np.vstack((train_5[:145,:], train_6[:200,:]))
N_train = np.size(x_train_brut,axis=0)
class_train = np.ones((345,1))   # label 1 for digit 6
class_train[:145] = 0       # label 0 for digit 5

x_test_brut = np.vstack((train_5[145:,:], train_6[200:,:]))
N_test = np.size(train_5,axis=0)+np.size(train_6,axis=0)-N_train

## Exercise 2: logistic regression to classify 5 & 6

1. Note that pixel values are between -1 and 1 by using the functions
 `min(I(:))` and `max(I(:))`.
2. Identify the indices of the most significant pixels, which are defined 
as having a standard deviation greater than 0.5 here. We denote by `lis_sig`
the list of positions of these significant pixels in the image vector.
_Indication : the function `std` gives the standard deviation (columnwise
in matrices) and you should find 173 pixel positions.
3. Show a binary image to locate these pixels.
_Indication : `Isig = zeros(16); Isig(list_sig)=1; Isig=Isig';`._
4. Define the training set `x_train` from `x_train_brut` from the significant pixels only.
5. Do the same with `x_test_brut` to extract `x_test`.
6. Use `regression_logistique.m` to estimate the logistic regression vector
`w` from the training set `x_train`. 
Choose `Nitermax = 13; eps_conv = 1e-3;`
7. Compute the decision function and the labels of the test set `x_test`. 
_Indication : do not forget the column of ones !_
8. Estimate the classification error rate by using :
`erreur = sum(abs(class-class_test))/N_test;`.
9. Locate some misclassified examples and visualize the corresponding image.
Comment on your results and observations.


In [ ]:
list_sig = np.where(np.std(x_train_brut, axis=0) > 0.5)[0].tolist()

Isig = np.zeros(256)
Isig[list_sig]=1
Isig = np.reshape(np.array(Isig), (16,16))
plt.imshow(Isig, cmap="gray")

## 3. Logistic regression using `scikit-learn`

1. **Go to** http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html for a presentation of the logistic regression model in `scikit-learn`.

2. **Apply** it to the present data set.

3. **Comment** on the use of logistic regression.

*Indication : you may have a look at* 

a) Theory : http://openclassroom.stanford.edu/MainFolder/DocumentPage.php?course=MachineLearning&doc=exercises/ex5/ex5.html

b) Video :  https://www.coursera.org/learn/machine-learning/lecture/4BHEy/regularized-logistic-regression 

c) Example : http://scikit-learn.org/stable/auto_examples/exercises/plot_digits_classification_exercise.html#sphx-glr-auto-examples-exercises-plot-digits-classification-exercise-py

*for a short presentation of regularized logistic regression.*

In [ ]:
# Include your code here
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(random_state=0, fit_intercept=True).fit(x_train, class_train)
w = clf.coef_[0]
bias = clf.intercept_[0]
print(f"Prédiction : {clf.predict(x_test)}")
print(f"Score : {clf.score(x_test, class_test)*100}%")
print(f"Coefficients de la régression : {bias, w}")


Grâce à la méthode de régression par scikit-learn, on trouve une fonction de régression de la forme :

$y = 2.63*x[0] - 2.12*x[1] + 5.46$